## Filter summary data

The ab_ag.tsv files contains multiple lines for each pdb_id, sometimes Hchain or Lchain are missing, etc.

This needs to be cleaned before one can process the data further.

Here we do a very simple cleaning step:

- remove lines where Hchain or Lchain are missing
- remove lines for which we do not have a PDB file
- for duplicated pdb_ids, select the first line only

You will probably need to do more selection of entries.
This is just to get started...

In [7]:
PDB_DIR = "../data/pdbs"


import pandas as pd
import os.path

summary = pd.read_csv('../data/ab_ag.tsv', sep='\t')

filtered_summary = summary[
    summary['Hchain'].notna() &
    summary['Lchain'].notna() &
    summary['pdb'].apply(lambda pdb: os.path.exists(f"{PDB_DIR}/{pdb}_chothia.pdb"))
].groupby('pdb').head(1)

Create a directory `generated/preprocess` and save the DataFrame there as `summary_pdb.tsv`.